### Import neccessary libaries

In [37]:
import sys, os, copy
from threading import Thread
import cv2, numpy as np


### Necessary functions

In [38]:
# OUTPUT_DIR = 'output'
# INPUT_DIR = 'input'

MAX_THREADS = 100
multi_thread = True
done_count = 0
total_count = 0
running_threads_count = 0

In [39]:
def replace_color(image, color_range_start, color_range_end, color_target):
    mask=cv.inRange(image,color_range_start,color_range_end)
    image[mask>0]=color_target
    return image

def get_img_line(img, range_color_start, range_color_end):
    blue_mask = cv2.inRange(img, range_color_start, range_color_end) # from 10 to 20
    line_img = cv2.bitwise_and(img,img, mask=blue_mask)
    return blue_mask, line_img

def create_img_line(img, bg_color, thick_line_color, line_thickness, all_cnts=None, blank_result=True):
    gray_line_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if blank_result:
        thick_line_img_gray = np.full_like(gray_line_img, 0)
        thick_line_img_rgb = np.full_like(img, bg_color)
    else:
        thick_line_img_gray = gray_line_img
        thick_line_img_rgb = img
    if all_cnts is None:
        all_cnts = cv2.findContours(gray_line_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = all_cnts[0] if len(all_cnts) == 2 else all_cnts[1]
    for c in cnts:
        cv2.drawContours(thick_line_img_gray, [c], -1, thick_line_color, thickness=line_thickness, lineType=cv2.FILLED)
        cv2.drawContours(thick_line_img_rgb, [c], -1, thick_line_color, thickness=line_thickness, lineType=cv2.FILLED)
    
    return all_cnts, thick_line_img_gray, thick_line_img_rgb

def fill_img_line_contours(img_gray, img_rgb, fill_color):
    laplacian = cv2.Laplacian(img_gray, cv2.CV_8UC1)
    lapl_bin, lapl_bin_val = cv2.threshold(laplacian, 25, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(lapl_bin_val, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    inpaint_mask = copy.deepcopy(img_rgb)
    for contour in contours:
        cv2.drawContours(inpaint_mask, [contour], -1, fill_color, thickness=-1)
    return inpaint_mask

def extract_segmentation_features(img_path, bg_color=[0, 255, 0], thick_line_color=[255,0,0], fill_color=[0,0,255], line_thickness=5, raise_error=False, post_function=None):
    global running_threads_count
    running_threads_count += 1

    img = cv2.imread(img_path)

    blue_mask, line_img = get_img_line(img, (250, 0, 0), (255, 20 ,255))

    blue_sum = np.sum(blue_mask)
    minimal_blue_count = 5
    if blue_sum <= minimal_blue_count:
        msg = f'img "{img_path}" has {blue_sum} blue sum while the minimum is {minimal_blue_count}'
        if raise_error:
            raise ValueError(msg)
        else:
            print(msg)
            running_threads_count -= 1
            return None     
    else:
        print('blue sum count:', blue_sum)   
    
    cnts, thick_line_img_gray, thick_line_img_rgb = create_img_line(line_img, bg_color, thick_line_color, line_thickness)

    # if len(cnts)==0:
    #     msg = f'img "{img_path}" has zero contours'
    #     if raise_error:
    #         raise ValueError(msg)
    #     else:
    #         print(msg)
    #         running_threads_count -= 1
    #         return None
    # else:
    #     print('countours', len(cnts))

    inpaint_mask = fill_img_line_contours(thick_line_img_gray, thick_line_img_rgb, fill_color)
    all_cnts, inpaint_thick_gray, inpaint_thick_rgb = create_img_line(inpaint_mask, bg_color, thick_line_color, line_thickness, all_cnts=cnts, blank_result=False)

    running_threads_count -= 1

    results = line_img, inpaint_mask, inpaint_thick_gray, inpaint_thick_rgb
    # results = line_img, thick_line_img_gray, thick_line_img_rgb, inpaint_mask
    if post_function is None:
        return results
    else:
        post_function(*results)



### Run

In [40]:
def iodir_process_helper(input_dir, output_dir='output', copy_input=True, segmentation_extraction_features_args=dict(), verbose=True):
    global done_count, total_count

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    print(f'input dir: ', input_dir)
    print(f'output dir: ', output_dir)
    
    for root, dirs, files in os.walk(input_dir, topdown=True):
        # print('file count:', len(files))
        file_count = len(files)
        done_count = file_count
        for i, file_name in enumerate(files):
            file_name:str = file_name
            input_img_path = os.path.join(root, file_name)
            output_img_dir = os.path.join(output_dir, file_name)
            if verbose:
                print(f'{i}/{file_count}: "{input_img_path}"')      
            
            segmentation_features = extract_segmentation_features(input_img_path, **segmentation_extraction_features_args)
            done_count += 1
            
            if segmentation_features is None:
                continue                  
            
            # segmentation_features = [cv2.resize(img_i, (224, 224)) for img_i in segmentation_features]

            base_file_name, extention_file_name = file_name.split('.', 1)
            if not os.path.isdir(output_img_dir):
                os.mkdir(output_img_dir)

            line_img, inpaint_mask, inpaint_thick_gray, inpaint_thick_rgb = segmentation_features
            mask_123 = replace_color(cv2.cvtColor(inpaint_thick_gray, cv2.COLOR_GRAY2RGB), (1,1,1), (1,1,1), (2,2,2))
            mask_123 = replace_color(mask_123, (255,255,255), (255,255,255), (3,3,3))
            mask_123 = replace_color(mask_123, (128,128,128), (128,128,128), (1,1,1))

            cv2.imwrite(os.path.join(output_img_dir, base_file_name+f'_line.png'), line_img)
            cv2.imwrite(os.path.join(output_img_dir, base_file_name+f'_inpaint_mask.png'), inpaint_mask)
            cv2.imwrite(os.path.join(output_img_dir, base_file_name+f'_inpaint_thick_gray.png'), inpaint_thick_gray)
            cv2.imwrite(os.path.join(output_img_dir, base_file_name+f'_inpaint_thick_rgb.png'), inpaint_thick_rgb)
            cv2.imwrite(os.path.join(output_img_dir, base_file_name+f'_mask123.png'), mask_123)

            # cv2.imwrite(os.path.join(output_dir, base_file_name+f'_filled_line.png'), filled_mask)

            if copy_input:
                with open(input_img_path, 'rb') as iip_fd:
                    with open(os.path.join(output_img_dir, file_name), 'wb+') as oop_fd:
                        oop_fd.write(iip_fd.read())

        # for name in dirs:
        #     print(os.path.join(root, name))



CLASSIFIED_TARGET_DIRS = ['Lingkar Mulut (Labeling)']
all_threads = []
for ctd in CLASSIFIED_TARGET_DIRS:
    print('dir: ', ctd)
    iohelper_kwargs = dict(input_dir=os.path.join('dentistsmile_tfds', 'dataset', ctd), output_dir='segmentation_true_mask', segmentation_extraction_features_args={
        'bg_color': [1, 1, 1], 
        'thick_line_color':[255,255,255], 
        'fill_color':[128, 128, 128],
        'line_thickness':25, 
        'raise_error': False
    })
    thread = Thread(target=iodir_process_helper, kwargs=iohelper_kwargs)
    all_threads.append(thread)
    thread.start()

for i, thread in enumerate(all_threads):
    print(f'Waiting thread {i}/{len(all_threads)} to finish')
    thread.join()

print('Done')
    


dir:  Lingkar Mulut (Labeling)
Waiting thread 0/1 to finish
input dir:  dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)
output dir:  segmentation_true_mask
0/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0363D.jpg"
blue sum count: 4999020
1/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0022A.jpg"
blue sum count: 4644825
2/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0145C.jpg"
blue sum count: 5153040
3/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0167E.jpg"
blue sum count: 5680890
4/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0280B.jpg"
blue sum count: 1244145
5/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0187B.jpg"
blue sum count: 1805910
6/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0256E.jpg"
blue sum count: 3284910
7/2194: "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0006E.jpg"
img "dentistsmile_tfds/dataset/Lingkar Mulut (Labeling)/0006E.jpg" has 0 blue sum while the minimum is 5
